# visualize video


In [ ]:
import cv2
import numpy as np
import os
from PIL import Image
def save_episode_video(episode_data, key, output_dir):
    """Save the episode video to a file."""
    frames = []

    # Collect frames from episode data
    rgb_data = episode_data['episode_data']['camera_info']['pano_camera_0']['rgb']
    for frame in rgb_data:
        # Convert the frame to a PIL image and then to a NumPy array
        pil_image = Image.fromarray(frame)
        frames.append(np.array(pil_image))

    # Define output video file path
    output_file = os.path.join(output_dir, f"episode_video_{key}.mp4")

    # Check the dimensions of the first frame
    if len(frames) > 0:
        height, width, layers = frames[0].shape
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(output_file, fourcc, 6, (width, height))

        # Write frames to video
        for frame in frames:
            video_writer.write(frame)

        # Release the video writer
        video_writer.release()
        print(f"Video saved successfully to {output_file}")
    else:
        print("No frames to save to video.")



# 生成数据集 in docker/


In [1]:
import json
import os
data_split = 'val_unseen'
json_file = f"vlmaps/docker/valid_paths/{data_split}_PReval_gather_data.json"

with open(json_file, 'r') as f:
    data = json.load(f)
    episode_all = []
for item in data:
    traj = data[item]
    for t in traj:
        print(t['episode_id'])
        episode_all.append(t['episode_id'])

print(len(episode_all))
# dump episode into unseen.txt
main_dir = f"vlmaps/docker/valid_paths/{data_split}"
if not os.path.exists(main_dir): 
    os.makedirs(main_dir)   
txt_file = os.path.join(main_dir, "episode_ids.txt")
with open(txt_file, 'w') as f:
    for episode in episode_all:
        f.write(f"{episode}\n")


1
2
3
13
14
15
25
26
27
37
38
39
40
41
42
55
56
57
73
74
75
85
86
87
124
125
126
136
137
138
145
146
147
154
155
156
163
164
165
235
236
237
247
248
249
250
251
252
253
254
255
352
353
354
361
362
363
367
368
369
415
416
417
418
419
420
439
440
441
445
446
447
505
506
507
544
545
546
571
572
573
628
629
630
655
656
657
670
671
672
673
674
675
712
713
714
715
716
717
742
743
744
745
746
747
820
821
822
826
827
828
862
863
864
865
866
867
868
869
870
892
893
894
910
911
912
913
914
915
922
923
924
934
935
936
955
956
957
967
968
969
1015
1016
1017
1024
1025
1026
1033
1034
1035
1039
1040
1041
1069
1070
1071
1099
1100
1101
1108
1109
1110
1135
1136
1137
1165
1166
1167
1195
1196
1197
1198
1199
1200
1240
1241
1242
1261
1262
1263
1285
1286
1287
1321
1322
1323
1333
1334
1335
1354
1355
1356
1375
1376
1377
1384
1385
1386
1387
1388
1389
1435
1436
1437
1438
1439
1440
1474
1475
1476
1477
1478
1479
1480
1481
1482
1519
1520
1521
1522
1523
1524
1612
1613
1614
1645
1646
1647
1678
1679
1680
1681
1682
168

In [2]:

import gzip
import json
import copy
import os
starrt_pos_dict = {}
episode_id = []
trajectory_id_list = []

target_dir = f'vlmaps/docker/valid_paths/{data_split}'

object_path_file = os.path.join(target_dir, "object.txt")
object_id_file = os.path.join(target_dir, "object_id.txt")
room_path_file = os.path.join(target_dir, "room.txt") 
room_id_file = os.path.join(target_dir, "room_id.txt")
with open(object_path_file, 'w') as f:
    pass
with open(object_id_file, 'w') as f:
    pass
with open(room_path_file, 'w') as f:
    pass
with open(room_id_file, 'w') as f:
    pass
def get_sub_trajectory_id_list():
    with open(txt_file, 'r') as f:
        sub_trajectory_id_list = [int(line.strip()) for line in f.readlines()]
        # 重新按大小排序
        sub_trajectory_id_list.sort()
    return sub_trajectory_id_list


def find_gpu_id(trajectory_id,num_of_gpus,trajectory_id_list):
    for idx, id in enumerate(trajectory_id_list):
        if (id == trajectory_id):
            return idx % num_of_gpus
def judge_stair_exists(path):
    '''
    path_z: ndarray, if max(path_z)- min(path_z) > 1.5, then there is a stair
    '''
    path_z = []
    for path in path:
        path_z.append(path[1])
    return max(path_z)- min(path_z) > 1.5

def load_data(split,num_of_gpus):
    ''' Load data based on VLN-CE
    '''
    # dataset_root_dir = base_data_dir # '../VLN/VLNCE/R2R_VLNCE_v1-3'
    dataset_root_dir = '/ssd/xiaxinyuan/code/VLN/VLNCE/R2R_VLNCE_v1-3'
    scene_id_list = []
    total_scans = []
    load_data = []
    split_data = [[] for _ in range(num_of_gpus)]
    trajectory_id_list = get_sub_trajectory_id_list()
    print(len(trajectory_id_list))
    for split in splits:
            with gzip.open(os.path.join(dataset_root_dir, f"{split}", f"{split}.json.gz"), 'rt', encoding='utf-8') as f:
                data = json.load(f)
            for item in data["episodes"]:
                scene_id = item['scene_id'].split('/')[1]
                if scene_id not in scene_id_list:
                    scene_id_list.append(scene_id)
            for item in data["episodes"]:
                if not judge_stair_exists(item['reference_path'])  and (item['trajectory_id'] in trajectory_id_list):
                    instruction = item['instruction']['instruction_text']
                    scene_id = item['scene_id'].split('/')[1]
                    if not ('room' in instruction or 'hall' in instruction or 'way' in instruction):
                        with open(object_path_file, 'a') as f:
                            f.write(f"{scene_id},{item['trajectory_id']},{item['episode_id']},{item['instruction']['instruction_text']}\n")
                        with open(object_id_file, 'a') as f:
                            f.write(f"{item['episode_id']}\n")
                    else:
                        with open(room_path_file, 'a') as f:
                            f.write(f"{scene_id},{item['trajectory_id']},{item['episode_id']},{item['instruction']['instruction_text']}\n")
                        with open(room_id_file, 'a') as f:
                            f.write(f"{item['episode_id']}\n")
                    

    return split_data
    # return load_data, list(set(total_scans))
# splits = ['train', 'val_seen', 'val_unseen', 'test']
splits = ['val_unseen']
# splits = ['val_seen']
# splits = ['val_seen']
num_of_gpus = 4
for split in splits:
    split_data = load_data(split,num_of_gpus)
    print(split_data)
    


954
[[], [], [], []]


In [10]:
a = [1,2,3]
print(a[4:])

[]


In [1]:
description_txt_file = 'vlmaps/docker/valid_paths/sub_success_paths_room_1212.txt'
target_txt_file = 'vlmaps/docker/valid_paths/sub_success_paths_id_1212.txt'
with open(description_txt_file, 'r') as f:
    for line in f.readlines():
        # if the line can be seperated by space with more than 2 parts, then it is a valid line
        if len(line.split()) > 2:
            # write the line to the target file, seperate by ','
            if  len(line.split()[0])!=11:
                continue
            with open(target_txt_file, 'a') as f:
                f.write(line.split()[2]+'\n')
                print(line.split()[2])


2
28
68
69
90
92
100
102
115
127
128
129
172
217
247
248
249
358
359
407
445
599
600
609
626
733
745
749
840
894
919
926
929
960
962
980
987
988
1048
1050
1071
1073
1078
1082
1085
1100
1135
1163
1215
1239
1242
1243
1257
1263
1264
1338
1364
1366
1390
1391
1441
1442
1444
1468
1541
1543
1802
1839
1840
1864
1910
1916
1923
1930
1947
1997
2010
2011
2038
2138
2145
2172
2173
2200
2209
2210
2237
2343
2354
2356
2410
2428
2441
2442
2443
2461
2507
2558
2595
2627
2628
2629
2682
2708
2709
2717
2735
2736
2737
2776
2819
2960
2962
2964
3009
3034
3044
3100
3173
3228
3308
3310
3338
3339
3340
3367
3377
3378
3379
3413
3414
3435
3436
3507
3508
3546
3573
3599
3600
3618
3635
3636
3637
3678
3685
3798
3806
3807
3808
3839
3882
3885
3891
3892
4022
4037
4039
4075
4169
4170
4248
4289
4348
4394
4445
4447
4451
4466
4467
4468
4524
4525
4604
4605
4606
4676
4677
4732
4806
4850
4851
4861
4871
4872
4873
4967
4998
4999
5006
5007
5008
5013
5014
5037
5039
5040
5041
5085
5086
5118
5122
5124
5131
5137
5140
5213
5215
5219
5220


In [6]:
from vlmaps.application_my.utils import check_valid_parsed_instruction
check_valid_parsed_instruction("self.get_pos('wall')")


True

In [5]:

import pytest

def test_valid_instructions():
    """测试有效的指令"""
    valid_instructions = [
        "self.turn(90)",
        "self.turn(90.5)",
        "self.turn(-180)",
        "self.move_forward(1.5)",
        "self.move_to_object('chair')",
        'self.move_to_object("table")',
        "self.move_to_left('wall')",
        "self.move_to_right('door')",
        "self.move_in_between('desk', 'chair')",
        "self.face('window')",
        "self.turn_absolute(45.5)",
        "self.get_pos('sofa')",
        "self.get_contour('bed')",
        "self.with_object_on_left('cabinet')",
        "self.with_object_on_right('shelf')"
    ]
    
    for instruction in valid_instructions:
        assert check_valid_parsed_instruction(instruction) == True

def test_invalid_instructions():
    """测试无效的指令"""
    invalid_instructions = [
        # 无效的函数名
        "self.invalid_function(90)",
        "self.unknown_action('chair')",
        
        # 参数数量错误
        "self.turn()",
        "self.turn(90, 180)",
        "self.move_to_object()",
        "self.move_in_between('desk')",
        "self.move_in_between('desk', 'chair', 'table')",
        
        # 参数类型错误
        "self.turn('90')",  # 数字不应该用引号
        "self.turn(chair)", # 缺少引号的字符串
        "self.move_forward('fast')", # 应该是数字
        "self.move_to_object(123)", # 应该是字符串
        
        # 语法错误
        "self.turn(90",  # 缺少右括号
        "self.move_to_object'chair')", # 缺少左括号
        "turn(90)", # 缺少self
        
        # 引号不匹配
        "self.move_to_object('chair\")",
        "self.move_in_between('desk\", \"chair')",
    ]
    
    for instruction in invalid_instructions:
        try:
            check_valid_parsed_instruction(instruction)
        except Exception as e:
            print(e)

def test_edge_cases():
    """测试边界情况"""
    edge_cases = [
        # 特殊字符
        "self.move_to_object('chair-1')",
        "self.move_to_object('dining_table')",
        "self.move_to_object('kitchen/living room')",
        
        # 空格处理
        "self.turn( 90 )",
        "self.move_to_object( 'chair' )",
        "self.move_in_between( 'desk' , 'chair' )",
        
        # 小数点和负数
        "self.turn(-90.5)",
        "self.move_forward(.5)",
        "self.turn_absolute(-0.0)",
    ]
    
    for instruction in edge_cases:
        assert check_valid_parsed_instruction(instruction) == True

def test_parameter_combinations():
    """测试参数组合"""
    # 生成不同的数字组合
    numbers = [0, 1, -1, 0.5, -0.5, 90, 180, 360, -360]
    for num in numbers:
        assert check_valid_parsed_instruction(f"self.turn({num})") == True
        assert check_valid_parsed_instruction(f"self.move_forward({num})") == True
        assert check_valid_parsed_instruction(f"self.turn_absolute({num})") == True
    
    # 生成不同的字符串组合
    objects = ['chair', 'table', 'desk', 'sofa', 'bed', 'door', 'window']
    for obj in objects:
        assert check_valid_parsed_instruction(f"self.move_to_object('{obj}')") == True
        assert check_valid_parsed_instruction(f"self.face('{obj}')") == True
    
    # 测试两个参数的组合
    for obj1 in objects[:3]:
        for obj2 in objects[3:]:
            assert check_valid_parsed_instruction(f"self.move_in_between('{obj1}', '{obj2}')") == True


test_valid_instructions()
test_invalid_instructions()
test_edge_cases()
test_parameter_combinations()
print("所有测试通过！")

指令解析错误: 无效的函数名: invalid_function
指令解析错误: 无效的函数名: unknown_action
指令解析错误: 函数 turn 需要 1 个参数，但收到了 0 个
指令解析错误: 函数 turn 需要 1 个参数，但收到了 2 个
指令解析错误: 函数 move_to_object 需要 1 个参数，但收到了 0 个
指令解析错误: 函数 move_in_between 需要 2 个参数，但收到了 1 个
指令解析错误: 函数 move_in_between 需要 2 个参数，但收到了 3 个
指令解析错误: 参数 chair 应该是数字类型
指令解析错误: 参数 fast 应该是数字类型
指令解析错误: 参数 123 应该是字符串类型，不应该是数字
指令解析错误: list index out of range
指令解析错误: list index out of range
所有测试通过！


In [4]:
test_action_lists = ["self.move_to_object(1)", "self.move_to_object('1')", "self.move_to_object(1.0)", "self.move_to_object('1.0')"]
for action in test_action_lists:
    try:
        check_valid_parsed_instruction(action)
        print(f"valid action: {action}")
    except Exception as e:
        print(e)

指令解析错误: 函数 move_to_object 需要 1 个参数，但收到了 0 个
valid action: self.move_to_object('1')
指令解析错误: 函数 move_to_object 需要 1 个参数，但收到了 0 个
valid action: self.move_to_object('1.0')
